In [1]:
import cx_Oracle
import os
import csv
from math import log10
from operator import itemgetter, attrgetter

In [2]:
# os.environ["NLS_LANG"] = "POLISH_POLAND.AL32UTF8"
# connection = cx_Oracle.connect("C##PPAWLUK", "#Lex2018", "144.21.89.60:1521/ORCL.591263988.oraclecloud.internal")
os.environ["NLS_LANG"] = "POLISH_POLAND.AL32UTF8"
connection = cx_Oracle.connect("SYSTEM", "Cytryna96", "localhost:1521/ORCL")

In [25]:
cur = connection.cursor()

In [26]:
# cur.execute('DELETE FROM "C##PPAWLUK"."TEMAT_DOKUMENTU"')
cur.execute('DELETE FROM "SYSTEM"."TEMAT_DOKUMENTU"')
cur.execute('commit')

In [27]:
dokumentsName = cur.execute('select distinct nazwa_dokumentu from dokument')

In [28]:
names = []

In [29]:
for dokument in dokumentsName:
    names.append(dokument[0])

In [30]:
dokumentsText = []

In [31]:
j = 0
for name in names:
    dokumentsText.append({'name': name, 'text': ''})
    text = cur.execute('select tekst from dokument where nazwa_dokumentu = \'{}\''.format(name))
    for t in text:
        dokumentsText[j]['text'] = dokumentsText[j]['text'] + t[0]
    j = j + 1

In [32]:
allTekst = ''

In [33]:
for z in dokumentsText:
    allTekst = allTekst + z['text']

In [34]:
allTekst = allTekst.split(' ')
allTekst.remove('')
allTekst = list(set(allTekst))

In [35]:
matrix = []

In [36]:
for word in allTekst:
    for dokument in names:
        matrix.append({"name":dokument,"word": word,'frequency':0})

In [37]:
for dokument in dokumentsText:
    text = dokument['text']
    text = text.split(' ')
    text.remove('')
    for m in matrix:
        if(m['name'] == dokument['name']):
            m['frequency'] = text.count(m['word'])

In [3]:
def themaDoc(matrix,nameDoc):
    tmpMatrix = []
    sum = 0
    for m in matrix:
        if(m['name'] == nameDoc):
            sum = sum + m['frequency']
            tmpMatrix.append(m)
    sortMatrix = sorted(tmpMatrix, key=lambda x: x['frequency'],reverse=True)
    size = 15
    if( len(sortMatrix) < size):
        size = len(sortMatrix)
    tematy = ''
    for i in range(0,size,1):
        tematy = tematy + ' ' + sortMatrix[i]['word'] + ' ' + '{:.2f}%'.format((float(sortMatrix[i]['frequency'])/float(sum))*100)+','
        print("{} {:.2f}%".format(sortMatrix[i] ,(float(sortMatrix[i]['frequency'])/float(sum))*100))
#     cur.execute('INSERT INTO "C##PPAWLUK"."TEMAT_DOKUMENTU" (NAZWA_DOKUMENTU, TEMATY) VALUES( \'{}\', \'{}\')'.format(nameDoc,tematy))
    cur.execute('INSERT INTO "SYSTEM"."TEMAT_DOKUMENTU" (NAZWA_DOKUMENTU, TEMATY) VALUES( \'{}\', \'{}\')'.format(nameDoc,tematy))
    return sortMatrix

In [39]:
tmpM = []
for name in names:
    tmpM.append(themaDoc(matrix,name))

{'name': 'D20181943.txt', 'word': 'sztandar', 'frequency': 17} 3.60%
{'name': 'D20181943.txt', 'word': 'centymetr', 'frequency': 15} 3.18%
{'name': 'D20181943.txt', 'word': 'rozporządzenie', 'frequency': 14} 2.97%
{'name': 'D20181943.txt', 'word': 'załącznik', 'frequency': 13} 2.75%
{'name': 'D20181943.txt', 'word': 'strona', 'frequency': 12} 2.54%
{'name': 'D20181943.txt', 'word': 'płat', 'frequency': 11} 2.33%
{'name': 'D20181943.txt', 'word': 'kolor', 'frequency': 11} 2.33%
{'name': 'D20181943.txt', 'word': 'sprawa', 'frequency': 10} 2.12%
{'name': 'D20181943.txt', 'word': 'pozycja', 'frequency': 10} 2.12%
{'name': 'D20181943.txt', 'word': 'numer', 'frequency': 10} 2.12%
{'name': 'D20181937.txt', 'word': 'postępowanie', 'frequency': 10} 6.06%
{'name': 'D20181937.txt', 'word': 'dzień', 'frequency': 8} 4.85%
{'name': 'D20181937.txt', 'word': 'ustawa', 'frequency': 8} 4.85%
{'name': 'D20181937.txt', 'word': 'artykuł', 'frequency': 7} 4.24%
{'name': 'D20181937.txt', 'word': 'karny', 'fr

In [40]:
for i in tmpM:
    print(i)

[{'name': 'D20181943.txt', 'word': 'sztandar', 'frequency': 17}, {'name': 'D20181943.txt', 'word': 'centymetr', 'frequency': 15}, {'name': 'D20181943.txt', 'word': 'rozporządzenie', 'frequency': 14}, {'name': 'D20181943.txt', 'word': 'załącznik', 'frequency': 13}, {'name': 'D20181943.txt', 'word': 'strona', 'frequency': 12}, {'name': 'D20181943.txt', 'word': 'płat', 'frequency': 11}, {'name': 'D20181943.txt', 'word': 'kolor', 'frequency': 11}, {'name': 'D20181943.txt', 'word': 'sprawa', 'frequency': 10}, {'name': 'D20181943.txt', 'word': 'pozycja', 'frequency': 10}, {'name': 'D20181943.txt', 'word': 'numer', 'frequency': 10}, {'name': 'D20181943.txt', 'word': 'państwowy', 'frequency': 9}, {'name': 'D20181943.txt', 'word': 'dzień', 'frequency': 7}, {'name': 'D20181943.txt', 'word': 'wewnętrzny', 'frequency': 7}, {'name': 'D20181943.txt', 'word': 'minister', 'frequency': 7}, {'name': 'D20181943.txt', 'word': 'złoty', 'frequency': 7}, {'name': 'D20181943.txt', 'word': 'jednostka', 'freque

In [41]:
cur.execute('commit')

In [42]:
cur.close()
connection.close()

In [43]:
f = open('Macierz.csv', 'w', newline='')
with f:
    fnames = ['word'] + names
    writer = csv.DictWriter(f, fieldnames=fnames,delimiter=';')
    writer.writeheader()
    matrix = []
    for m in tmpM:
        for i in m:
            matrix.append(i)
    matrix = sorted(matrix, key=lambda k: k['word'])
    tab = {}
    for m in matrix:
        tab.__setitem__(m['name'],m['frequency'])
        if(matrix.index(m)%len(names) == len(names)-1):
            tab.__setitem__('word',m['word'])
            writer.writerow(tab)
            tab.clear()
            

In [50]:
f = open('TFIDF.csv', 'w', newline='')
with f:
    fnames = ['word'] + names
    writer = csv.DictWriter(f, fieldnames=fnames,delimiter=';')
    writer.writeheader()
    matrix = []
    for m in tmpM:
        for i in m:
            matrix.append(i)
    matrix = sorted(matrix, key=lambda k: k['word'])
    tab = {}
    liczbaDokumentowWystapieniaSlowa = 0
    liczbaDokumentow = len(names)
    for m in matrix:
        if(m['frequency']>0):
            tab.__setitem__(m['name'],m['frequency'])
            liczbaDokumentowWystapieniaSlowa = liczbaDokumentowWystapieniaSlowa + 1
        else:
            tab.__setitem__(m['name'],0)
        if(matrix.index(m)%len(names) == len(names)-1):
            IDF = log10(float(liczbaDokumentow)/float(liczbaDokumentowWystapieniaSlowa))
            for name in names:
                tab.__setitem__(name,float(tab.get(name))*IDF)
            tab.__setitem__('word',m['word'])
            writer.writerow(tab)
            tab.clear()
            liczbaDokumentowWystapieniaSlowa = 0